# MolSim 2020: ML for Gas Adsorption

In this exercise we will build a model that can predict the CO$_2$ uptake of MOFs.

There are two main **learning goals** for this exercise: 
1. Understand the typical workflow for machine learning in materials science. We will cover exploratory data analysis (EDA), unsupervised learning (PCA) and supervised learning (KRR)
2. Learn about some Python packages that are useful for data analysis and visualization. If you are not already familiar with Python, this exercise can be a great opportunity to learn some basics

For some of the exercises we assume some basic knowledge of Python, e.g. that you can write list comprehensions,  understand the documentation of packages or the docstrings of the functions. You will usually need to provide some function arguments (we indicate the places with 'fillme' comments)

If you struggle with this, click on the provided hints and/or partner up with someone more experienced. 

## Import the packages we will need

In [1]:
# basics 
import os 
import numpy as np 

# data
import pandas as pd 
import pandas_profiling
from pandas_profiling.config import config
config.config.set_file('config_report.yaml') # use our custom configuration

# machine learning 
# scaling of data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# train/test split
from sklearn.model_selection import train_test_split
# model selection 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# model
from sklearn.kernel_ridge import KernelRidge
# pipeline 
from sklearn.pipeline import Pipeline
# PCA
from sklearn.decomposition import PCA
# Dummy model
from sklearn.dummy import DummyClassifier, DummyRegressor
# Variance Threshold 
from sklearn.feature_selection import VarianceThreshold
# metrics 
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             mean_absolute_error, mean_squared_error, max_error)
# feature names
from descriptornames import * 

# save/load models 
import joblib

# For the permutation importance
from joblib import Parallel
from joblib import delayed
from sklearn.metrics import check_scoring
from sklearn.utils import Bunch
from sklearn.utils import check_random_state
from sklearn.utils import check_array


# plotting 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns

# for interactive plots, you can try to use holoviewes
import holoviews as hv
from holoviews import dim, opts
hv.extension('plotly', 'bokeh', 'matplotlib')

RANDOM_SEED = 4242424242
DATA_DIR = 'data'
DATA_FILE = os.path.join(DATA_DIR, 'data.csv')

np.random.seed(RANDOM_SEED)


ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.7 from "/Users/leopold/Applications/miniconda3/envs/molsim_ml/bin/python",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.17.4" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error 

 $\color{DarkBlue}{\textsf{Short question}}$
- We declared a global variable to fix the random seed. Why did we do this?  

Below, we define two global variables (hence upper case), which are the names of our feature and target columns. 

In [ ]:
TARGET = 'pure_uptake_CO2_298.00_15000' #pure_uptake_CO2_298.00_1600000
TARGET_BINARY = 'target_binned'
FEATURES = (geometric_descriptors + summed_functionalgroup_descriptors 
            + summed_linker_descriptors + summed_metalcenter_descriptors)

As descriptors we will use geometry properties such as density, pore volume ... and [revised autocorrelation functions](https://pubs.acs.org/doi/abs/10.1021/acs.jpca.7b08750) that encode metal center, linkers and the functionalgroups by calculating heuristics on the structure graph.

$\color{DarkBlue}{\textsf{Short question}}$

We use descriptors that encode pore geometry properties (density, pore diameters, surface areas) as well as some that describe the chemistry of the MOF (the RACs). 
- Would you expect the feature importance to be different for high and low pressure gas adsorption?

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> <a href="https://pubs.acs.org/doi/abs/10.1021/acs.chemmater.8b02257">An article from Diego et al.</a> (10.1021/acs.chemmater.8b02257) gives some hints.</li>
</ul>
</details>

## Import the data

In [ ]:
df = pd.read_csv(DATA_FILE)

Below, we will just give it a quick look to make sure that everything looks fine. 
Try to answer the following question.

 $\color{DarkBlue}{\textsf{Short question}}$
- How many materials are there? 
- Which datatypes do we deal with?

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> use the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html"><code>df.info</code></a> method</li>
</ul>
</details>


In [ ]:
df.info() ## add code here

Let's take a look at the first few rows if everythings seems reasonable

In [ ]:
df.head()

### Split the data

Before doing any analysis or transformation on the data, we split it into two disjoint sets. 
If you want more explanation why this is important, you might want to look into [chapter 7.10.2 of Elements of Statistical Learning](https://web.stanford.edu/~hastie/ElemStatLearn//printings/ESLII_print10.pdf). In a nutshell, we want to avoid *any* data leakage, also in terms of the feature selection.

#### Split with stratification

If there are imbalanced classes, we want to make sure that random sampling does not distort class distributions. 
I.e., if we would have only very few good materials in our dataset, we might have nearly none of them in our test set if we are unlucky in our random sampling. 

[Stratification](https://en.wikipedia.org/wiki/Stratified_sampling) ensures that the class distributions (ratio of good to bad materials) is the same in the training and test set.

Later, we will explore the effect of this stratification in more detail.


For stratification to work, we need categories. Currently, our target is continuos, i.e. we have to binarize if. 
As a threshold we will use---following [Boyd et al.](https://www.nature.com/articles/s41586-019-1798-7)--2 mmol CO$_2$ / g for the target property. We will also use these to classes for the classification exercises in which we try to classify a material as 'high' or 'low' performing.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
 - create a column that encodes using 0 (low performing) and 1 (high perfoming) to encode the category which a material belongs to.

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> you can use <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html'>pd.cut</a>, 
    <a href='https://stackoverflow.com/questions/4406389/if-else-in-a-list-comprehension'>list comprehension</a>, the <a href='https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Binarizer.html#sklearn.preprocessing.Binarizer'> binarizer in sklearn </a>...) </li>
</ul>
</details>

In [ ]:
THRESHOLD = 2 # mmol CO2/g
df['target_binned'] =  # add your code

Now, we can perform the actual split

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- select reasonable values for XX and XY and then perform the test/train splits. What do you consider when making this decision? 

In [ ]:
df_train_stratified, df_test_stratified = train_test_split(df, train_size=XX, 
                                                                test_size=XY, 
                                                                random_state=RANDOM_SEED, 
                                                                stratify=df['target_binned']) 

#### Split without stratification (random split)

In [ ]:
df_train, df_test = train_test_split(df, train_size=XX, 
                                         test_size=XY, 
                                         random_state=RANDOM_SEED) 

## Exploratory data anaylsis 

Now, as we are sure that we have put data aside that we won't touch, we can give it a closer look

### Get some description of the data

Use the [`pandas_profiling`](https://github.com/pandas-profiling/pandas-profiling) package to get an overview over the data.

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Are there skewed distributions? 
- Are there missing values? 
- What are other issues with the dataset that you might to fix before training a model?
- How could one fix those issues?

Note that the function calculates many statistics by default, which can take some time. For this reason, we use our own configuration file, in which we turned some of the calculations of for the first step. 

*You can speed it up even more by subsampling the data*

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> use <code>df.sample(500)</code> to get a random sample of 500 rows, e.g. `df_train_stratified.sample(500)`. </li>
    <li> the function call might then look like `pandas_profiling.ProfileReport(df_train_stratified.sample(500))` </li>
</ul>
</details>



In [ ]:
pandas_profiling.ProfileReport()

### Correlations

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Plot some features against the target properties and calculate the Pearson and Spearman correlation coefficient
- What are the strongest correlations? 
- Are they different for the different targets and does this correspond to what you would expect?

### Visualization

In [ ]:
df_train.shape

The output of `df_train.shape` will show you that our data is high-dimensional. 
It's hard to visualize such data.

$\color{DarkBlue}{\textsf{Short Exercise}}$
- How many materials are in the training set?
- How many features do we use?

#### PCA 

One way to visualize high dimensional data is to use [principal component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) which baiscally tries to find linear combinations of features that explain most of the variance in the data.

One can then plot the data in two dimensions in terms of the two first principal components.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- plot the explained variance as a function of the number of principal components and plot a scree plot. Use all features for this analysis (on the training set).
- plot the dataset in terms of the first two principal components. What are the features that are most important in those first principal components (look at the loadings of the first two principal components). 

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> A scree plot shows the explained variance as a function of the number of components. For this, we run <code>PCA(n_components=n_features)</code>, where <code>n_components=X.shape[1] </code> </li>
    <li> The <code> PCA </code> object also has a <code> fit </code> method </li>
    <li> To access the explained variance, you can use the <code>explained_variance_ratio_ </code> attribute of the object</li>
    <li> For plotting, you can use <code> 
        data = {
            'feature' : np.arange(0, len(FEATURES)),
            'explained variance ratio': # fill code
        }
        hv.Curve(data, 'feature', 'explained variance ratio')
        </code> </li>
</ul>
</details>

In [ ]:
# Build the PCA objects and run it 
pca = PCA(#fill)
pca.fit(#fill)

In [ ]:
# Plot the scree plot

#### PCA on Swiss Roll  

Some data comes in interesting shapes. The Swiss roll dataset was created to test out 
dimensionality reduction algorithms by creating some data in 2D and mapping it then to 3D  using a smooth function (here: $(x,y,z) := (x \cos (x), y, x \sin(x))$).

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Create some Swiss roll data (by coding the Gaussian mixture in 2D and the mapping to 3D yourself). Visualize the data in 2D and then in 3D. You need to use the proper function arguments to do this. 

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> Both <code>means</code> and <code>covariances</code> take lists as input.  </li>
    <li> The list for the mean contains tuples of (x,y) coordinates for the centers of the Gaussian blobs and the list for the covariances takes floats. </li>
    <li> try e.g. <code> means = [(-2, -2), (-2,2), (2,-2), (2,2)], covariances = [1,1,1,1] </code> </li>
    <li> for plotting in holoviews you can for example use 
        <code>scatter = hv.Scatter(gaussian_mix, 'x', ['y', 'color']).opts(color='color', cmap='rainbow')
        scatter
        </code>
    </li>
    <li> for plotting in matplotlib you can use the <a href="https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.scatter.html"><code>plt.scatter</code></a>. You can use the <code>c</code> argument to color the points.
</ul>
</details>

In [ ]:
def gaussian_mixture_data(means: list=[(-2, -2), (1,1)], 
                          covariances: list=[1], numpoints: int=200) -> pd.DataFrame:
    """
    generate some gaussian mixture data
    """
    from sklearn.datasets import make_gaussian_quantiles
    assert len(means) == len(covariances)
    
    X = []
    Y = []
    c = []
    
    # loop over all means and covariances
    for i, mucov in enumerate(zip(means, covariances)):
        mu, cov = mucov
        print(f'making blob of mean {mu} and covariance {cov}')
        x, y = make_gaussian_quantiles(mean=mu, cov=cov,
                                         n_samples=numpoints, 
                                         random_state=RANDOM_SEED)
        X.append(x[:,0])
        Y.append(x[:,1])
        c.append([i  + 1] * numpoints)
    
    
    gaussian_mix = {
        'x': np.array(X).flatten(),
        'y': np.array(Y).flatten(),
        'color': np.array(c).flatten() 
    }
    
    return pd.DataFrame.from_dict(gaussian_mix)

In [ ]:
gaussian_mix = gaussian_mixture_data(##fillme)

##### Plot it 

In [ ]:
# plot it

##### Map it to 3D and plot it

In [ ]:
def map_to_3d(x: float, y:float) -> list: 
    """
    x cos (x), y, x sin(x)
    Feel free to play aroung with it
    """
    return [.5 * x * np.cos(x), y,   2 * x * np.sin(x)]

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Add the correct arguments (instead of ????) to the function call in the list comprehension.
- Plot the result in three dimensions using the 'color' column for coloring

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> For the 3D plot you can for example use <a href='http://holoviews.org/reference/elements/matplotlib/Scatter3D.html'><code>Scatter3D</code></a>. Note that this function currently does not work with the <a href='http://dev.holoviews.org/Tutorials/Bokeh_Backend.html'>Bokeh backend</a> </li>
    <li> The plotting code code for example look like 
        <code>
        scatter3d = hv.Scatter3D(gaussian_mix, kdims=['x', 'y'], vdims=['z', 'color'])
        scatter3d = scatter3d.opts(
            opts.Scatter3D(color='color',  cmap='rainbow'))
        scatter3d
        </code>
    </li>
</ul>
</details>

In [ ]:
coordinates = np.array([map_to_3d(#fillme) for x,y in zip(gaussian_mix['x'], gaussian_mix['y'])])

gaussian_mix['x'] = coordinates[:,0]
gaussian_mix['y'] = coordinates[:,1]
gaussian_mix['z'] = coordinates[:,2]

In [ ]:
# Plot the data in 3d

##### Run the PCA transformation

Now, we can run the transformation on the training data. 
I.e., project the data from feature space onto the first two principal components and then plot the data in two dimension in these coordinates.

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Project the three dimensional swiss roll data onto the first two principal components
- Plot the data in two dimensions, use the original coloring (which could e.g. represent different material classes)


<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> To perform the the transformation, you can use the <code>fit_transform()</code> method of the <code>PCA(n_components=2)</code> instance</li>
    <li> To plot the data you can use the <code>Scatter</code> method from holoviews.</li>
    <li> If you struggle with plotting, you can modify the following code 
    <code>        
    swissroll_transformed_dict = {}
    swissroll_transformed_dict['x'] = swissroll_transformed[:,#fill with a integer]
    swissroll_transformed_dict['y'] = swissroll_transformed[:,#fill with a integer]
    swissroll_transformed_dict['color'] = gaussian_mix['color']
    scatter = hv.Scatter(swissroll_transformed_dict, 'x', ['y', 'color']).opts(
        color='color', cmap='rainbow')
    scatter
    </code>
    </li>
</ul>
</details>



In [ ]:
# run the PCa
pca = PCA(#fillme)
swissroll_transformed = pca.fit_transform(#fillme)

In [ ]:
# plot it

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- If you look at the 2D-plot, what would you have expected a dimensionality reduction to do? What did actually happen? 
- What happens to the transformation if you scale one feature? What does it mean in practice when you e.g. want to understand the importance of features in terms of their variance?

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> To standardize one variable you can use <code>gaussian_mix['x'] = (gaussian_mix['x']- gaussian_mix['x'].mean())/gaussian_mix['x'].std()</code> </li>
    <li> Or just try to multiply by a constant </li>
    <li> For practical application, give a look to the data in the dataframe ... it might come in different units ... </li>
</ul>
</details>


## Baselines

For machine learning, it is important to get some *baselines* to which one then compare the results of a model. 

In practice, one can use really simple models, the state of the art, or simple heuristics to do this. We will use latter here.

For this we use `Dummy` objects that only calculate the mean, the median or the most frequent case on the training set (when you run the `fit()` method) and then use this to perform the predictions.

### Build dummy models

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Create three [`DummyClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html) and [`DummyRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html) instances for `uniform`, `stratified` and `most_frequent` voting (classification) and `mean`, `median`. (e.g. `dummyinstance = DummyClassifier(strategy='')`)
- Train them on the training data (`dummymodel.fit(df_train[FEATURES], df_train[TARGET_BINARY]`) with and without stratification

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> to create <code>DummyClassifier</code> you can for example use <code> dummyclassifier_uniform = DummyClassifier(strategy='uniform') </code> </li>
    <li> to fit it, you can use the  <code> classifier.fit(X, y) </code> method
    <li> classification and regression use the same X (anyway doesn't matter here) but different y!</li>
</ul>
</details>

In [ ]:
# Build DummyClassifiers

In [ ]:
# Build DummyRegressors

#### Evaluate the performance of the dummy models

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Calculate precision, reall, accuracy and the F1 score for the dummy classifiers. What would you expect those numbers to be? Do the actual values surprise you and what does this mean in practice when one reports metrics?
- Calculate maximum error, mean absolute error and mean square error for the dummy regressors.

It can be handy to store the metrics in a nested dictionary: 

```python
{'dummyestimator1': {
                        'metric_a_key': metric_a_value, 
                        'metric_b_key': metric_b_value
                    },
 'dummyestimator2': {
                        'metric_a_key': metric_a_value, 
                        'metric_b_key': metric_b_value
                    },
 }
``` 

Addionally, it can also be handy to write functions that returns all the metrics and that you can for example call with `get_classification_metrics(model, X, y_true)` and that then returns a dictionary with the metrics as key-value pairs. Below we give two functions that does that, you only need to fill some arguments to make it work.

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> to perform a prediction using a estimator object, you can call <code> classifier.predict(X) </code> </li>
    <li> to calculate metrics, you can for example call <code>accuracy_score(true_values, predicted_values) </code> </li>
    <li> it can be useful to write a small function that returns all the metrics
        <code> 
        def get_classification_metrics(model, X, y_true):
            y_predicted = model.predict(X)
            accuracy = accuracy_score(y_true, y_predicted)
            f1_score = f1_score(y_true, y_predicted)
            recall = recall_score(y_true, y_predicted)
            precision = precision_score(y_true, y_predicted)
            metrics_dict = {
                'accuracy': accuracy, 
                'f1_score': f1_score, 
                'recall': recall,
                'precision': precision
            }
            return metrics_dict
        </code>
        <code> 
        def get_regression_metrics(model, X, y_true): 
            y_predicted = model.predict(X)
            mae = mean_absolute_error(y_true, y_predicted)
            mse = mean_squared_error(y_true, y_predicted)
            maximum_error = max_error(y_true, y_predicted)
            metrics_dict = {
                'mae': mae, 
                'mse': mse, 
                'max_error': maximum_error
            }
            return metrics_dict
        </code>
    </li>
    <li> to create a nested dictionary, it can be useful to store the dummy models in a squence of tuples <code>[('dummy_a', dummy_a_instance), ('dummy_b', dummy_b_instance)] 
          then, you can just loop over this and use the first elements of the tuples as keys of the nested dictionary.
        </code>
    </li>
  <li>
      This list of tuples can for example look like
      <code>
dummy_regressors = [
    ('mean', dummyregressor_mean), 
    ('median', dummyregressor_median)
]
dummy_classifiers = [
    ('uniform', dummyclassifier_uniform), 
    ('stratified', dummyclassifier_stratified),
    ('majority', dummyclassifier_majority)
]
      </code>
  </li>
 <li>
     
 </li>
</ul>
</details>

In [ ]:
def get_classification_metrics(model, X, y_true):
    """
    Get a dicionary with classification metrics:
    model: sklearn model with predict method
    X: feature matrix
    y_true: ground truth labels
    """
    y_predicted = model.predict(#fillme)
    
    # use the true values and the predicted values as arguments to
    # calculate the scores
    accuracy = accuracy_score(#fillme)
    f1 = f1_score(#fillme)
    recall = recall_score(y_true, y_predicted)
    precision = precision_score(y_true, y_predicted)
    
    metrics_dict = {
        'accuracy': accuracy, 
        'f1_score': f1, 
        'recall': recall,
        'precision': precision
    }
    
    return metrics_dict

In [ ]:
def get_regression_metrics(model, X, y_true): 
    """
    Get a dicionary with classification metrics:
    model: sklearn model with predict method
    X: feature matrix
    y_true: ground truth labels
    """
    y_predicted = model.predict(#fillme)
    
    mae = mean_absolute_error(#fillme)
    mse = mean_squared_error(#fillme)
    maximum_error = max_error(#fillme)
    
    metrics_dict = {
        'mae': mae, 
        'mse': mse, 
        'max_error': maximum_error
    }
    
    return metrics_dict

## Build actual regression models

Some issues you might noticed with the data are that 
- The data is not properly scaled (different features might be measured in different units ...)
- Some features do not contain much information, e.g., are constant
- Some feature distributions are skewed (which is more relevant for some models than for others ...)
- There are only few good materials (also the target distribution is skewed), so we might have problems predicting well for the best materials

In this first iteration, we do not care about those. We just want to get familiar with Kernel Ridge Regression (KRR) and train a model on all features without any optimization.

You can try different kernels, but we recommend to start with Gaussian kernels ('rbf')


 $\color{DarkBlue}{\textsf{Short Question}}$
- Do you expect this model to perform better than the dummy models?
- Train it and then calculate the performance metrics on the train and test set.?

In [ ]:
# Train the model
krr = KernelRidge(#fillme)
krr.fit(#fillme)

In [ ]:
# get the metrics on the train and the test set

Of course, one shouldn't use default model parameters and one usually should perform some feature engineering.
We will do this in the following.

## Improve the model

### Scaling and building a first pipeline.

Given that we will now start doing more than just training a model, we will build [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), which are objects that can be used to collect a selection of transformations and estimators.

This makes it quite easy to apply the same set of operations on different datasets.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Build a pipline that first performs standard scaling and then a KRR. Call it `pipe_basic`. 
- Train it using the stratifyied training data and calculate the performance metrics (max error, MAE, MSE) on the training and test set

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
   <li> the syntax to build a pipeline is as follows (list of tuples): 
       <code> pipe_w_scaling = Pipeline([
       ('name1', Transformer()),
       ('name2': Estimator())
   ]) </code> </li>
    <li> the KRR instance can be built using <code> KRR() </code>, the scaler instance using <code> StandardScaler() </code>
    <li> the <code>fit</code>, <code>predict</code> methods also work for pipelines </li>
</ul>
</details>

In [ ]:
pipe_w_scaling = Pipeline(
   # fillme with the pipeline steps
)

In [ ]:
# fit the pipeline

In [ ]:
# get the metrics

### Adding more steps to the pipeline

The model is still bad. 
The next thing we want to optimize is to remove features with low variance, we can do this by using a variance threshold.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Add a variance threshold to the pipline
- Retrain the pipeline and calculate the performance metrics (max error, MAE, MSE) on the training and test set

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> The variance theshold is a <a href='https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html'> transformer built into the sklearn library</a>. You can call <code>VarianceThreshold</code> </li>
</ul>
</details>

In [ ]:
# Define the pipeline
pipe_variance_threshold = Pipeline(
    # fillme with the pipeline steps
)

In [ ]:
# Fit the pipeline and run the evaluation
pipe_variance_threshold.fit(#fillme)

### Hyperparameter optimization

Given that we now have a decent feature matrix---but still a very bad model---we can procede with optimizing the model. 
The most basic way for doing this is grid search. In which we define a grid of parameters for which we want to evaluate the model performance.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Think about which parameters you could optimize in the pipeline, remember that you can also switch of some steps by setting them to `None` or e.g. use a different scaling method. Also, the model has some parameters you might want to optimize.
For each parameter you need to define a resonable range of parameters.
- Run the hyperparameter optimization using 5-fold cross-validation (you can adjust the number of folds according to your computational resources/impatience. It turns out at k=10 is the [best tradeoff between variance and bias, though](https://arxiv.org/abs/1811.12808)). Tune the hyperparameters until you are statisfied (e.g., until you cannot improve the cross validated error any more)
- Why don't we use the test set for hyperparameter tuning? 
- Evaluate the model performance by calculating the performance metrics (MAE, MSE, max error) on the training and the test set.
- Instead of grid search, try to use random search on the same grid (`RandomizedSearchCV`) and fix the number of evaluations to a fraction of the number of evaluations of grid search. What do you observe and conclude?

<details>
<summary> <font color='green'>Click here for some more information about hyperparameter optimization</font></summary>
Note that using grid search is not the best way to perform hyperparamter optimization. Even [random search was shown to be more efficient](http://jmlr.csail.mit.edu/papers/volume13/bergstra12a/bergstra12a.pdf). Really efficient though are Bayesian optimization approaches like [TPE](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf). This is implemented in the hyperopt library, which we also installed in the conda enviornment.
</details>

<details>
<summary> <font color='green'>Click here for hyperparameter optimization with hyperopt (outlook)</font></summary>
    
<b>Import the tools we need</b>
<code>
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, mix, rand, anneal, space_eval
from functools import partial
</code>    

<b>Define the grid</b>
<code>
param_hyperopt = {
    "krr__alpha": hp.loguniform("krr__alpha", np.log(0.001), np.log(10)),
    "krr__gamma": hp.loguniform("krr__gamma", np.log(0.001), np.log(10)),
}
</code> 

<b>Define the objective function</b>
<code>
def objective_function(params):
    pipe.set_params(
        **{
            "krr__alpha": params["krr__alpha"],
            "krr__gamma": params["krr__gamma"],
        }
    )
    score = cross_val_score(
        pipe, X_train, y_train, cv=10, scoring="neg_mean_absolute_error"
    ).mean()
    return {"loss": -score, "status": STATUS_OK} 
</code>

<b>We will use a search in which we mix random search, annealing and tpe</b>
<code>
trials = Trials()
mix_search = partial(
   mix.suggest,
   p_suggest=[(0.15, rand.suggest), (0.15, anneal.suggest), (0.7, tpe.suggest)],
)
</code>

<b>Now, we can minimize the objective function.</b>
<code>
best_param = fmin(
        objective_function,
        param_hyperopt,
        algo=mix_search,
        max_evals=MAX_EVALES,
        trials=trials,
        rstate=np.random.RandomState(RANDOM_SEED),
    )
</code>

</details>

 $\color{DarkRed}{\textsf{Tips}}$
- If you want to see what is happening, set the verbosity argument of the `GridSearchCV` object to a higher number
 
- If you want to speed up the optimization, you can run it in parallel by setting the `n_jobs` argument to the number of workers. If you set it to -1 it will use all available cores.
 
- If the optimization is too slow, reduce the number of datapoints, the number of folds or the grid size. Note that it can also be a feasible strategy to first use a coarser grid and the a finer grid for finetuning.

<details>
<summary> <font color='green'>Click here for hints about pipelines and grid search</font></summary>
<ul>
    <li> You can use the <code>np.logspace</code> function to generate grid for values that you want to vary on a logarithmic scale </li>
    <li>For grid search, you need to define a parameter grid, which is a dictionary of the following form: 
        <code>
            param_grid = {
                    'pipelinestage__parameter': np.logspace(-4,1,10),
                    'pipelinestage': [None, TransformerA(), TransformerB()]
            }
        </code>
    </li> 
    <li>The grid search object itself looks like <code>GridSearchCV(pipe, param_grid=param_grid, cv=number_folds)</code>. It also has a <code>fit</code> method.</li>
</ul>
</details>

In [ ]:
# Define the parameter grid and the grid search object
param_grid = {
    # fillme
}

grid_krr = GridSearchCV(#your pipeline, param_grid=param_grid, 
                        cv=#number of folds, verbose=2, n_jobs=-1)

random_krr = RandomizedSearchCV(#your pipeline, param_distributions=param_grid, n_iter=,
                        cv=#number of folds, verbose=2, n_jobs=-1)

In [ ]:
# run the grid search by calling the fit method 
grid_krr.fit(#fillme)
random_krr.fit(#fillme)

In [ ]:
# get the performance metrics

## Saving the model

Now, that we spent so much time in optimizing our model, we do not want to loose it. 

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- use the [joblib library](https://scikit-learn.org/stable/modules/model_persistence.html) to save your model
- make sure you can load it again

Use this to save models as you go.

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li>Use the <code>dump</code> method to save the model and the <code>load</code> method to load it again</li>
</ul>
</details>

In [ ]:
# Dump your model

In [ ]:
# Try to load it again

## Evaluate the model performance in detail.

Now, lets investigate the performance of the model in more detail---beyond simple numeric scores.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- for the training and test data, plot a parity plot (true values against predictions)
- plot a histogram of the distribution of the training and test errors on the training and test set, plot the errors also as a function of the true value
- for a setting in which we would the ML for prescreening purposes (i.e. to screen a library of millions of porous materials and to find those with the highest gas uptake) what kind of errors would you tolerate? Could you tolerate the errors of your model?

For this exercise, it can be handy to save the results in a dictionary, e.g. 
```(python)
res_train = {
    'y true': ,
    'y pred': 
}
```

As you will need to run this multiple times, it can be useful to make a function that can create such a dictionary.

<details>
<summary> <font color='green'>Click here for hints for plotting</font></summary>
<ul>
    <li> If you want to use matplotlib, you can use the <a href="https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist2d.html">hist2d function</a> </li>
    <li> If you want to use holoviews (interactive plots) you can start using the following snippet. Note that you need to use the bokeh backend here.
        <code> 
            hv.extension('bokeh')
            hex_train = hv.HexTiles(res_train, ['y true', 'y pred']).hist(dimension=['y true','y pred'])
            hex_test = hv.HexTiles(res_test, ['y true', 'y pred']).hist(dimension=['y true', 'y pred'])
            hex_train + hex_test
        </code>
    </li>
    <li>to create the frequencies and the edges of a histogram, one can use <code>np.histogram</code></li>
</ul>
</details>

In [ ]:
# Create dictionaries with training and test results 
res_train = {
    'y true': # fillme
    'y pred': # fillme
}

res_test = {
    'y true': # fillme
    'y pred': # fillme
}

In [ ]:
res_train['error'] = res_train['y true'] - res_train['y pred']
res_test['error'] = res_test['y true'] - res_test['y pred']

In [ ]:
# plot it

## Influence of Regularization

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- what happens if you set $\alpha=0$ or to large value? Why is this the case?
 To test this, fix this value in one of your pipelines (you might want to re-optimize the other hyperparameters), retrain the models and rerun the performance evaluation.

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> Check the derivation for ridge regression and KRR in the notes. </li>
</ul>
</details>

## Interpreting the model

Now, that our model performs decently, we would like to now what features are resposible for this, i.e. how the model performs it's reasoning. 

One method to do so is the [permutation feature importance technique](https://christophm.github.io/interpretable-ml-book/feature-importance.html).

Note that also this method is not a silver bullet, e.g. there are issues with correlated features, but it is likely [a better choice than feature importance, like impurity decrease, derived from random forestes](https://explained.ai/rf-importance/).

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Complete the function `_calculate_permutation_scores` (which we took from the `sklearn` package) and which is needed to calculate the permutation feature importance using the `permutation_importance` function. 

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> If you struggle to build the function, try to complete the following code:
    <code> def </code>
    </li>
    <li> If you still struggle, you can use the pre-built function from the <a href='http://rasbt.github.io/mlxtend/user_guide/evaluate/feature_importance_permutation/'> MlXtend library </a>
</ul>
</details>

In [ ]:
def _calculate_permutation_scores(estimator, X, y, col_idx, random_state,
                                  n_repeats, scorer):
    """Calculate score when `col_idx` is permuted. Based on the sklearn implementation
    
    estimator: sklearn estimator object
    X: pd.Dataframe or np.array
    y: pd.Dataframe or np.array
    col_idx: int
    random_state: int
    n_repeats: int 
    scorer: function that takes model, X and y_true as arguments
    """
    random_state = check_random_state(random_state)

    X_permuted = X.copy()
    scores = np.zeros(n_repeats)
    # get the indices
    shuffling_idx = np.arange(X.shape[0])
    for n_round in range(n_repeats):
        # shuffle them (fill in what you want to shuffle)
        random_state.shuffle(#fillme)  
        
        # Deal with dataframes
        if hasattr(X_permuted, "iloc"):
            # .iloc selects the indices from a dataframe and yougive it [row, column]
            col = X_permuted.iloc[#fillme]
            col.index = X_permuted.index
            X_permuted.iloc[:, col_idx] = col
            
        # Deal with numpy arrays 
        else:
            # array indexing is [row, column]
            X_permuted[:, col_idx] = X_permuted[#fillme]
        
        # Get the scores
        feature_score = scorer(estimator, X_permuted, y)
        
        # record the scores in array 
        scores[n_round] = feature_score

    return scores

In [ ]:
def permutation_importance(estimator, X, y, scoring=None, n_repeats=5,
                           n_jobs=-1, random_state=None):
    """Permutation importance for feature evaluation 
    estimator : object
        An estimator that has already been :term:`fitted` and is compatible
        with :term:`scorer`.
    X : ndarray or DataFrame, shape (n_samples, n_features)
        Data on which permutation importance will be computed.
    y : array-like or None, shape (n_samples, ) or (n_samples, n_classes)
        Targets for supervised or `None` for unsupervised.
    scoring : string, callable or None, default=None
        Scorer to use. It can be a single
        string (see :ref:`scoring_parameter`) or a callable (see
        :ref:`scoring`). If None, the estimator's default scorer is used.
    n_repeats : int, default=5
        Number of times to permute a feature.
    n_jobs : int or None, default=None
        The number of jobs to use for the computation.
        `None` means 1 unless in a :obj:`joblib.parallel_backend` context.
        `-1` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.
    random_state : int, RandomState instance, or None, default=None
        Pseudo-random number generator to control the permutations of each
        feature. See :term:`random_state`.
    """
    # Deal with dataframes
    if not hasattr(X, "iloc"):
        X = check_array(X, force_all_finite='allow-nan', dtype=None)

    # Precompute random seed from the random state to be used
    # to get a fresh independent RandomState instance for each
    # parallel call to _calculate_permutation_scores, irrespective of
    # the fact that variables are shared or not depending on the active
    # joblib backend (sequential, thread-based or process-based).
    random_state = check_random_state(random_state)
    random_seed = random_state.randint(np.iinfo(np.int32).max + 1)
     
    # Determine scorer from user options.
    scorer = check_scoring(estimator, scoring=scoring)
    # get the performance score on the unpermuted data 
    baseline_score = scorer(estimator, X, y)
    
    # run the permuted evaluations in parallel for each column
    scores = Parallel(n_jobs=n_jobs)(delayed(_calculate_permutation_scores)(
        estimator, X, y, col_idx, random_seed, n_repeats, scorer
    ) for col_idx in range(X.shape[1]))
    
    # get difference two
    importances = baseline_score - np.array(scores)
    
    # return the results (dictionary)
    return Bunch(importances_mean=np.mean(importances, axis=1),
                 importances_std=np.std(importances, axis=1),
                 importances=importances)

In [ ]:
permutation_results = permutation_importance(#fillme)

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Use your function to find the five most important features. Would you expect this result?

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> To get the top <emph>n</emph> indices of an array <code>a</code>, you can use <code> np.argsort(a)[-n:]</code></li>
    <li> Get the feature names from the <code>FEATURES</code> list </li> 
</ul>
</details>

## Additional research questions

- Change the target from CO$_2$ to methane. Is it easier to predict? What changes in the feature importance?
- Do also a search over model space, do you find better performance with a Gradient Boost model? 